## Testing the pipeline as we go

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import modules
import sys
import os
import pandas as pd
import numpy as np


# import from local data files
current_path = os.path.abspath('.')
sys.path.append(os.path.dirname(current_path))

from data_access.data_factory import DataFactory as factory
from utils import config as cf
from utils import data as data
from utils import model as md

from functools import reduce

#####################################
#### PARAMETERS FROM CONFIG FILE ####
#####################################


''' Number of different combinations of grid search hyperparameters
Default is 500, use a lower value, >=1 to speed-up the evaluations
Will only be used when regression with regularisation model is used
for making predictions'''
    
# Only required if model uses regularisation
parm_spce_grid_srch=cf.parm_spce_grid_srch

# Create a list of alphas to cross-validate against
alphas_val = cf.alphas_val


#When should the weekly training start from
strt_training_period=cf.chsen_datum

#Flag to set if one chooses Zero inflated regression model

'''By default, the flag is reset as the architecture of
the model is more suited for tranches of low prevalence
when there are excessively large number of LSOAs with zero
reported cases. This architecture is not suitable for longer
time periods where the dataset is less likely to have excessive
number of zero reported cases
'''

zero_inf_flg_st=cf.zero_infltd_modl

'''By default, the flag is reset which means
regression with regularisation model will be
used for making predictions (sklearn package). Do note: irrespective 
of the status of the flag, the significant coefficients 
will be through linear regression model (statsmodel package)
'''
lin_regr_or_regrlsn=cf.linear_rgr_flg

################
### INGESTS ####
################

# static variables
print("Ingesting static data....")
static_df = factory.get('static_vars_for_modelling').create_dataframe()

# LSOA Area information to normalise footfall
print("Ingesting area data....")
area_df = factory.get('static_subset_for_norm').create_dataframe()

# Cases data
print("Ingesting cases data....")
cases_df = factory.get('aggregated_tests_lsoa').create_dataframe()

# Mobility data
print("Ingesting mobility data....")
deimos_footfall_df = factory.get('lsoa_daily_footfall').create_dataframe()

Ingesting static data....
Ingesting area data....
Ingesting cases data....
Ingesting mobility data....


Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

In [60]:


##############################
#### PROCESS STATIC DATA #####
##############################

### DROP THE FEATURES THAT THE CORRELATION ANALYSIS SHOWED THAT WE DON'T NEED
### FILTER FOR NUMERIC COLUMNS ONLY
### AGE COLUMNS ARE DROPPED

static_df_num=static_df.set_index(['LSOA11CD','travel_cluster'])
static_df_num=static_df_num.select_dtypes(include=np.number)
static_df_num=static_df_num[[x for x in static_df_num.columns if x!='Area']]


### SELECT THE REQUIRED COLUMNS ###

sel_colmns_set = ['CENSUS_2011_ASIAN_ASIAN_BRITISH','METHOD_OF_TRAVEL_TO_WORK_NON_MOTORISED','METHOD_OF_TRAVEL_TO_WORK_Public_TRANSPORT',\
                'METHOD_OF_TRAVEL_TO_WORK_WORK_MAINLY_FROM_HOME','FAMILIES_WITH_DEPENDENT_CHILDREN_NO_DEPENDENT_CHILDREN',\
                'care', 'meat_and_fish_processing','ready_meals', 'textiles', 'warehousing']


static_df_num=static_df_num[sel_colmns_set]

assert static_df_num.shape == (cf.n_lsoa, 10)


##############################
#### PROCESS AREA DATA #####
##############################

norm_area_df=area_df[['LSOA11CD','ALL_PEOPLE','Area']]

#### JOIN TO STATIC DATA ####
static_df_num=static_df_num.reset_index()
static_df_num=static_df_num.merge(norm_area_df,on=['LSOA11CD'],how='inner')

### NORMALISE ###

# Normalise the IDBR variables: 
'''expressed as number of residents per unit area
who are working in high risk industries'''
fctr=1
idbr_norm='Area'
static_df_num['care']=(static_df_num['care'].div(static_df_num[idbr_norm]))*(fctr)
static_df_num['meat_and_fish_processing']=(static_df_num['meat_and_fish_processing'].div(static_df_num[idbr_norm]))*(fctr)
static_df_num['textiles']=(static_df_num['textiles'].div(static_df_num[idbr_norm]))*(fctr)
static_df_num['ready_meals']=(static_df_num['ready_meals'].div(static_df_num[idbr_norm]))*(fctr)
static_df_num['warehousing']=(static_df_num['warehousing'].div(static_df_num[idbr_norm]))*(fctr)

static_df_num.drop(columns=['ALL_PEOPLE','Area'],inplace=True)

static_df_num.set_index(['LSOA11CD','travel_cluster'],inplace=True)

assert static_df_num.shape == (cf.n_lsoa, 10)

######### IMPLEMENT THE RESULTS OF THE EXPLORATORY FACTOR ANALYSIS ############


'''Based on Factors obtained earlier 
we combine features as follows'''

risk_ftrs=['meat_and_fish_processing', 'textiles', 'ready_meals','warehousing','care']

# features except high risk industry features
sep_ftrs=[x for x in static_df_num.columns if x not in risk_ftrs]
 

df_sep=static_df_num[sep_ftrs].reset_index()

df_risk=static_df_num[risk_ftrs].reset_index()


'''df_risk_sum captures 'interactions'
between IDBR features
'''

df_risk_sum=df_risk.select_dtypes(include=object)

### CREATE THE NEW COLUMNS

df_risk_sum.loc[:,'care_homes_warehousing_textiles']=df_risk[['LSOA11CD', 'travel_cluster','textiles', 'warehousing', 'care']].sum(axis=1)

df_risk_sum.loc[:,'meat_and_fish_processing']=df_risk[['LSOA11CD', 'travel_cluster','meat_and_fish_processing']].sum(axis=1)

df_risk_sum.loc[:,'ready_meals']=df_risk[['LSOA11CD', 'travel_cluster','ready_meals']].sum(axis=1)

### MERGE THE SEPARATE FEATURES WITH THE NEW RISK FEATURES

list_dfs=[df_sep,df_risk_sum]

static_df_new_variables = reduce(lambda left,right: pd.merge(left,right,on=['LSOA11CD','travel_cluster']), list_dfs)

# This is the final static dataframe - still neeed to add cases and dynamic data
static_df_new_variables=static_df_new_variables.merge(norm_area_df,how='inner',on=['LSOA11CD']).reset_index(drop=True)

assert static_df_new_variables.shape == (cf.n_lsoa, 12)


#################################
#### PROCESS THE CASES DATA #####
#################################

# sort values by date
cases_df_datum=cases_df[['Date','LSOA11CD','COVID_Cases']].sort_values(by='Date').reset_index(drop=True)
    
# create a list of dataframes of cases, one df for each week 
cases_df_datum=[pd.DataFrame(y) for x, y in cases_df_datum.groupby('Date', as_index=False)]
    
cases_df_datum_mrgd=[]

# The cases dataframe is split into different dates.
# This splitting allows for each dataframe to be left joined to the static data
# Therefore there will be a cases value for every LSOA for every week
# If no cases data is present for a given week in a given LSOA, the 'Date'
# field is filled with the 'Date' value from that DataFrame


# for each df
for splt_df in cases_df_datum:
    
    # store the date for the given DataFrame
    datm=splt_df['Date'].unique()[0]
    
    # left-join cases onto the static data
    df=static_df_new_variables.merge(splt_df,how='left',on=['LSOA11CD'])
    
    # fill any gaps in the cases data with the correct date
    df['Date']=df['Date'].fillna(datm)
    
    # any dates that needed to be filled had zero cases for that week
    df['COVID_Cases']=df['COVID_Cases'].fillna(0)
    
    # apply normalisation
    df['COVID_Cases']=df['COVID_Cases'].div(df['Area'])
    
    cases_df_datum_mrgd.append(df)
        
# stack the dataframes        
df_all_tranches_sbset=pd.concat(cases_df_datum_mrgd).reset_index(drop=True)

# drop the area column
df_all_tranches_sbset.drop('Area', axis=1, inplace=True)

# rename to reflect normalisation
df_all_tranches_sbset.rename(columns={'COVID_Cases':'COVID_Cases_per_unit_area'},inplace=True)

assert df_all_tranches_sbset.shape == (2627520, 13)



##########################################
#### CHOOSE THE MODEL TRAINING PERIOD ####
##########################################


# Change this filter if different time period 
# is required to infer respective risk predictors

df_all_tranches_sbset=df_all_tranches_sbset[df_all_tranches_sbset['Date']>=strt_training_period]

#### GENERATE A 'WEEK' COLUMN ###

date_list=sorted(df_all_tranches_sbset['Date'].dt.date.unique())
    
week_list=["week_"+str(x+1) for x in range(len(date_list))]
    
date_dict=dict(zip(date_list,week_list))
    
df_all_tranches_sbset['week']=df_all_tranches_sbset['Date'].map(date_dict)

df_all_tranches_sbset['Date']=df_all_tranches_sbset['Date'].astype(str)

df_all_tranches_sbset=df_all_tranches_sbset.reset_index(drop=True)

# This is to visually check we have same number of LSOAs in consecutive weeks of training data
print('Unique LSOAs in various training weeks {}'.format(df_all_tranches_sbset.groupby('Date')['LSOA11CD'].count().unique()))

assert df_all_tranches_sbset.groupby('Date')['LSOA11CD'].count().unique() == cf.n_lsoa


##############################
### PROCESS MOBILITY DATA ####
##############################

deimos_footfall_df['Date']=deimos_footfall_df['Date'].astype(str)

# Issue: December-2021: CJ
# Since the delay in cases data available to us is longer than the 
# delay in regular ingestion of mobility data- we use the 'excess'
# mobility data alongside the static predictors to predict the number
# of cases--This is one way to validate the results

# Dataset containing both the static and dynamic predictors alongside the target variable
df_all_tranches_sbset=df_all_tranches_sbset.merge(deimos_footfall_df,how='inner',on=['LSOA11CD','Date'])

# This is to visually check we have same number of LSOAs in consecutive weeks of training data (including both static and dynamic predictors)
# We should not lose any LSOAS: this value should be 32844
assert df_all_tranches_sbset.groupby('Date')['LSOA11CD'].count().unique() == cf.n_lsoa


#### CREATE TEST SET #####

# These are the dates for which we have mobility data but we don't have cases data

# Test (unseen data) for predicting future cases: we only capture the predictors in this dataset
df_all_tranches_sbset_tst_data=deimos_footfall_df.merge(static_df_new_variables,how='inner',on=['LSOA11CD'])

# Test dataset should contain timestamps not present in the training data
df_all_tranches_sbset_tst_data=df_all_tranches_sbset_tst_data[df_all_tranches_sbset_tst_data['Date']>df_all_tranches_sbset['Date'].max()].reset_index(drop=True)

assert df_all_tranches_sbset_tst_data.groupby('Date')['LSOA11CD'].count().unique() == cf.n_lsoa

#Test data date range
tst_dat_rng=df_all_tranches_sbset_tst_data['Date'].min()+'-'+df_all_tranches_sbset_tst_data['Date'].max()

# Drop columns we don't need



########################################
#### PROCESS THE DATA INTO TRANCHES ####
########################################

tranches_uk = cf.tranches_uk
events = cf.events

splt_df_tranches=[]

for tim_slice in range(len(tranches_uk)):
    print(tim_slice)
    
    # if tim_slice is the final element of the list
    if tim_slice == len(tranches_uk)-1:
        
        # t1 is the selected date
        t1 = tranches_uk[tim_slice]
        
        # subset for all dates after t1
        df_tim = df_all_tranches_sbset[df_all_tranches_sbset['Date']>t1]
        
        # if dataframe is not empty
        if df_tim.shape[0] != 0:
            
            # add a column for the event description
            df_tim['tranche_desc']=events[tim_slice]
        
        splt_df_tranches.append(df_tim)
    
    # if tim_slice is not the final element of the list
    else:
        
        # t1 is the selected date
        t1 = tranches_uk[tim_slice]
        
        # t2 is the next date in the list
        t2 = tranches_uk[tim_slice+1]
        
        # return the data between t1 and t2
        df_tim = data.split_time_slice(df_all_tranches_sbset,t1,t2)
        
        # if dataframe is not empty
        if df_tim.shape[0] != 0:
            
            # add a column for the event description
            df_tim['tranche_desc'] = events[tim_slice]
        
        splt_df_tranches.append(df_tim)
        
        

# remove sliced df for which there is no data available 
# (mobilty data is available from tranche 2 onwards, so the first sliced df will be empty)
splt_df_tranches=[x for x in splt_df_tranches if len(x)!=0]


# Perform aggregation of predictors and target variable for each tranche
# Each tranche contains multiple weeks, aggregation results in mean of each of the numerical features
# In practice, the static features are the same for each week, so we are averaging footfall over the tranche
# Each sliced df will have one unique record for each LSOA (because of averaging)
splt_df_tranches_agg=[]

for df_x in splt_df_tranches:
    
    # convert date column to string showing date range of the tranche
    df_x['Date']=str(df_x['Date'].min())+'-'+str(df_x['Date'].max())
    
    # define columns to group by
    grpp_colms=['Date','LSOA11CD','tranche_desc','travel_cluster']
    
    # compute the mean over each week in the tranche
    df_x=df_x.groupby(grpp_colms)[[x for x in df_x.columns if x not in grpp_colms]].mean().reset_index()
    
    # sort by LSOA code
    df_x=df_x.sort_values(by='LSOA11CD').reset_index(drop=True)
    
    splt_df_tranches_agg.append(df_x)
    

# stack each tranche into one dataframe
splt_df_tranches_conct=pd.concat(splt_df_tranches_agg).reset_index(drop=True)

# convert date to string
splt_df_tranches_conct['Date']=splt_df_tranches_conct['Date'].astype(str)

# find unique date range and tranche description combinations
df_key=splt_df_tranches_conct[['Date','tranche_desc']].drop_duplicates().reset_index(drop=True)

# put them into a dictionary
event_dict=dict(zip(df_key['Date'].values,df_key['tranche_desc'].values))

# list of integers from 1 to n_tranches inclusive 
tranche_order=list(range(1, cf.n_tranches + 1))

# zip the tranche descriptions and tranche numbers
event_order_dict=dict(zip(events,tranche_order))

# dict of tranche number: tranche description
rvse_event_dict={v: k for k, v in event_order_dict.items()}

# dict of tranche description: tranche date range
rvse_date_dict={v: k for k, v in event_dict.items()}

# the dataframe with all tranches
df_all_tranches_sbset=splt_df_tranches_conct

# create new column for tranche number
df_all_tranches_sbset['tranche_order']=df_all_tranches_sbset['tranche_desc'].map(event_order_dict)

# test that each LSOA appears one for each tranche
# there is no mobility data for the first tranch, therefore the shape should be (n_tranches - 1) * n_lsoa
assert df_all_tranches_sbset.shape == (((cf.n_tranches - 1) * cf.n_lsoa), 21)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Unique LSOAs in various training weeks [32844]
0
1
2


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:264: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



3
4
5
6


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:244: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:284: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [61]:
##################################
#### MORE MOBILITY PROCESSING ####
##################################

# COLUMNS TO DROP BEFORE MODELLING
cols_to_drop=['ALL_PEOPLE','msoa_people', 'resident_footfall_sqkm','total_footfall_sqkm','visitor_footfall_sqkm','worker_footfall_sqkm']

#Remove un-necessary columns from training and test data
df_all_tranches_sbset.drop(columns = cols_to_drop, inplace=True)
df_all_tranches_sbset_tst_data.drop(columns=cols_to_drop, inplace=True)



#convert mobility features into per sq metres and rename columns
fctr_mob = 0.000001

df_all_tranches_sbset[['worker_visitor_footfall_sqkm']] = df_all_tranches_sbset[['worker_visitor_footfall_sqkm']] * (fctr_mob)

df_all_tranches_sbset.rename(columns={'worker_visitor_footfall_sqkm':'worker_visitor_footfall_sqm'}
                                     , inplace=True)


df_all_tranches_sbset_tst_data[['worker_visitor_footfall_sqkm']] = df_all_tranches_sbset_tst_data[['worker_visitor_footfall_sqkm']] * (fctr_mob)

df_all_tranches_sbset_tst_data.rename(columns={'worker_visitor_footfall_sqkm':'worker_visitor_footfall_sqm'}
                                      ,inplace=True)


###############################
#### PROCESS IDBR FEATURES ####
###############################

#convert idbr features into per hectare (they were earlier expressed as per unit area (sq km): 1 sq km= 100 hectare)

fctr_risk = 0.01

df_all_tranches_sbset['care_homes_warehousing_textiles']=(df_all_tranches_sbset['care_homes_warehousing_textiles'])*(fctr_risk)
df_all_tranches_sbset['meat_and_fish_processing']=(df_all_tranches_sbset['meat_and_fish_processing'])*(fctr_risk)
df_all_tranches_sbset['ready_meals']=(df_all_tranches_sbset['ready_meals'])*(fctr_risk)


df_all_tranches_sbset_tst_data['care_homes_warehousing_textiles']=(df_all_tranches_sbset_tst_data['care_homes_warehousing_textiles'])*(fctr_risk)
df_all_tranches_sbset_tst_data['meat_and_fish_processing']=(df_all_tranches_sbset_tst_data['meat_and_fish_processing'])*(fctr_risk)
df_all_tranches_sbset_tst_data['ready_meals']=(df_all_tranches_sbset_tst_data['ready_meals'])*(fctr_risk)


# Aggregate test data-- so unique value for each LSOA
df_all_tranches_sbset_tst_data=df_all_tranches_sbset_tst_data.groupby(['LSOA11CD', 'travel_cluster'])[list(df_all_tranches_sbset_tst_data.\
                                                                                                           select_dtypes(include=np.number).columns)].mean().reset_index()



In [75]:
##########################
#### MODELLING BEGINS ####
##########################


# Create list of travel clusters
list_of_tc=sorted(df_all_tranches_sbset['travel_cluster'].unique())
'''Separate regression model is fit for each travel cluster- multi-group analysis
This is done to further account for any spatial correlation amongst the features'''

# Lists to store the outputs
str_pred_tc_static=[]
str_coef_tc_static=[]
str_se_coef_tc_static=[]
str_non_se_coef_tc_static=[]
str_pred_tc_recnt=[]

# for each travel cluster
for sbset_tc in list_of_tc:
    print(sbset_tc)
    
    # subset the training data for the travel cluster
    df_chsen=df_all_tranches_sbset[df_all_tranches_sbset['travel_cluster']==sbset_tc].reset_index(drop=True)
    
    # sort by tranche
    df_chsen=df_chsen.sort_values(by=['tranche_order','LSOA11CD']).reset_index(drop=True)
    
    # drop the columns of strings
    df_chsen=df_chsen[[x for x in df_chsen.columns if x not in ['tranche_desc','Date']]]
    
    # pass the training data, test data and parameters from the config file to the modelling function
    pred_tc,coef_tc,se_tc,non_se_tc,pred_tst_tc=md.fit_model_tranche_static_dynamic(lin_regr_or_regrlsn,df_chsen,zero_inf_flg_st,alphas_val,parm_spce_grid_srch,\
                                                                                    df_all_tranches_sbset_tst_data)
    
    # store the results
    str_pred_tc_static.append(pred_tc)
    str_coef_tc_static.append(coef_tc) 
    str_se_coef_tc_static.append(se_tc)
    str_non_se_coef_tc_static.append(non_se_tc)
    str_pred_tc_recnt.append(pred_tst_tc)



L1. >70% metropolitan core dwellers
CV starts without zero-inflated model and with regularisation
CV finishes without zero-inflated model and with regularisation
r2_score: train score=0.2748370896838259
CV starts without zero-inflated model and with regularisation
CV finishes without zero-inflated model and with regularisation
r2_score: train score=0.2769252389958712
CV starts without zero-inflated model and with regularisation
CV finishes without zero-inflated model and with regularisation
r2_score: train score=0.6877094909243264
CV starts without zero-inflated model and with regularisation
CV finishes without zero-inflated model and with regularisation
r2_score: train score=0.09265161878476358
CV starts without zero-inflated model and with regularisation
CV finishes without zero-inflated model and with regularisation
r2_score: train score=0.22214055908578623
CV starts without zero-inflated model and with regularisation
CV finishes without zero-inflated model and with regularisation
r

In [76]:
# Store most important features used for making predictions 
# for each travel cluster and for each tranche
# This includes both significant and non-significant features

# Model coefficients (with regularisation)
str_coef_tc_static=pd.concat(str_coef_tc_static).reset_index()

# Model predictions (with regularisation)
str_pred_tc_static=pd.concat(str_pred_tc_static).reset_index(drop=True)

# Store the standard error/p-value of the coefficients of trained model
# (No regularisation, standardised coefs)
str_se_coef_tc_static=pd.concat(str_se_coef_tc_static).reset_index()

# Store the standard error/p-value of the coefficients of trained model
# (No regularisation, non-standardised coefs)
str_non_se_coef_tc_static=pd.concat(str_non_se_coef_tc_static).reset_index()

# Store the predictions of trained model (on unseen test data)
pred_latest=pd.concat(str_pred_tc_recnt).sort_values(by='Predicted_cases_test',ascending=False).reset_index(drop=True)

pred_latest['Date']=tst_dat_rng

# Add  
pred_latest=pred_latest.merge(area_df,on=['LSOA11CD','travel_cluster','ALL_PEOPLE','Area'],how='inner').sort_values(by='Predicted_cases_test',ascending=False).reset_index(drop=True)

# Round negative predictions to zero
pred_latest['Predicted_cases_test']=pred_latest['Predicted_cases_test'].clip(lower=0)

LSOA with negative predictions on train data...
count    3440.000000
mean       -0.541308
std         1.294177
min       -32.786048
25%        -0.504810
50%        -0.200492
75%        -0.079751
max        -0.000040
Name: Predicted_cases_train, dtype: float64
LSOA with negative predictions on unseen test data...
count    577.000000
mean      -0.574884
std        0.801976
min       -8.755973
25%       -0.655099
50%       -0.316137
75%       -0.140799
max       -0.000633
Name: Predicted_cases_test, dtype: float64


In [77]:
output_dfs = [str_coef_tc_static, str_se_coef_tc_static, str_non_se_coef_tc_static]

for df in output_dfs:
    
    df['Date'] = df['tranche'].map(rvse_event_dict).map(rvse_date_dict)
    df['tranche_desc'] = df['tranche'].map(rvse_date_dict)
    df['Features'] = df['Features'].str.lower()

In [78]:
str_pred_tc_static

,Actual_cases,Predicted_cases_train,tranche_train,travel_cluster,LSOA11CD,Best_cv_score_train,RMSE_train,Probability_of_COVID_Case_train,Predicted_Class_train
0,5.281131,47.433315,2,L1. >70% metropolitan core dwellers,E01000001,0.303386,30.261590,0,0
1,6.064929,43.225596,2,L1. >70% metropolitan core dwellers,E01000002,0.303386,30.261590,0,0
2,47.371136,72.428699,2,L1. >70% metropolitan core dwellers,E01000003,0.303386,30.261590,0,0
3,2.350923,54.024138,2,L1. >70% metropolitan core dwellers,E01000005,0.303386,30.261590,0,0
4,13.914926,16.572683,2,L1. >70% metropolitan core dwellers,E01000842,0.303386,30.261590,0,0
...,...,...,...,...,...,...,...,...,...
197059,0.090226,-0.306116,7,L5. >70% rural dwellers,E01033529,0.701830,1.780165,0,0
197060,0.043312,0.138412,7,L5. >70% rural dwellers,E01033530,0.701830,1.780165,0,0
197061,5.520434,3.525779,7,L5. >70% rural dwellers,E01033538,0.701830,1.780165,0,0
197062,6.643441,4.450748,7,L5. >70% rural dwellers,E01033539,0.701830,1.780165,0,0


In [ ]:
# Inserting back the Date and other columns in the stored outputs

output_dfs = [str_coef_tc_static, str_se_coef_tc_static, str_non_se_coef_tc_static]

for df in output_dfs:
    
    df['Date'] = df['tranche'].map(rvse_event_dict).map(rvse_date_dict)
    df['tranche_desc'] = df['tranche'].map(rvse_date_dict)   
    df['Features'] = df['Features'].str.lower()


# For the prediction dataframe
str_pred_tc_static['tranche_desc']=str_pred_tc_static['tranche_train'].map(rvse_event_dict)
str_pred_tc_static['Date']=str_pred_tc_static['tranche_desc'].map(rvse_date_dict)


# For the prediction dataframes, remove the reference to training and rename coefficients
str_pred_tc_static.rename(columns={'tranche_train':'tranche'},inplace=True)

# For the non-regularisation model
str_se_coef_tc_static.rename(columns={'Coefficients':'coef_sklearn'},inplace=True)
str_se_coef_tc_static.rename(columns={'coef':'standardised_coef','P>|t|':'P_value','[0.025':'lower_bound','0.975]':'upper_bound','std err':'std_err'},inplace=True)

str_non_se_coef_tc_static.rename(columns={'P>|t|':'P_value','[0.025':'lower_bound','0.975]':'upper_bound','std err':'std_err'},inplace=True)

In [ ]:
#######################
#### WRITE RESULTS ####
#######################


# SAVE OUTPUTS
tranches=True
    
dataset_suffix = '_zir_static_dynamic'
if not zero_inf_flg_st:
    dataset_suffix = '_no' + dataset_suffix


    
if tranches:
    dataset_suffix=dataset_suffix+'_tranches'
else:
    dataset_suffix=dataset_suffix+'_no_tranches'   

# Regularisation coefs
str_coef_tc_static.to_gbq('review_ons.multi_grp_coef' + dataset_suffix, project_id='ons-hotspot-prod',if_exists='replace')

# Non-regularisation - standardised coefs
str_se_coef_tc_static.to_gbq('review_ons.multi_grp_se_coef' + dataset_suffix, project_id='ons-hotspot-prod',if_exists='replace')

# Non-regularisation - non-standardised coefs
str_non_se_coef_tc_static.to_gbq('review_ons.multi_grp_se_coef' + dataset_suffix, project_id='ons-hotspot-prod',if_exists='replace')

# Predictions for all tranches
str_pred_tc_static.to_gbq('review_ons.multi_grp_pred' + dataset_suffix ,project_id='ons-hotspot-prod',if_exists='replace')

# Predictions for the latest tranche only
pred_latest.to_gbq('review_ons.multi_grp_pred_test_data' + dataset_suffix,project_id='ons-hotspot-prod',if_exists='replace')

In [80]:
pred_latest

,LSOA11CD,Predicted_cases_test,travel_cluster,ALL_PEOPLE,Area,Date,RGN19NM,UTLA20NM,MSOA11NM,Country
0,E01032775,1409.258162,L1. >70% metropolitan core dwellers,1400,0.002385,2021-10-17-2021-12-12,London,Tower Hamlets,Tower Hamlets 031,England
1,E01002842,807.528281,L1. >70% metropolitan core dwellers,1012,0.004391,2021-10-17-2021-12-12,London,Kensington and Chelsea,Kensington and Chelsea 021,England
2,E01008506,712.356296,L3. >70% suburban dwellers,1379,0.012967,2021-10-17-2021-12-12,North East,North Tyneside,North Tyneside 012,England
3,E01013744,553.442305,L3. >70% suburban dwellers,1738,0.026219,2021-10-17-2021-12-12,East Midlands,Leicester,Leicester 018,England
4,E01033487,547.451733,L1. >70% metropolitan core dwellers,1491,0.007451,2021-10-17-2021-12-12,London,Islington,Islington 011,England
...,...,...,...,...,...,...,...,...,...,...
32839,E01023044,0.000000,L4. >70% exurban dwellers,1132,1.702108,2021-10-17-2021-12-12,South East,Hampshire,New Forest 023,England
32840,E01031621,0.000000,L4. >70% exurban dwellers,1683,1.863036,2021-10-17-2021-12-12,South East,West Sussex,Horsham 012,England
32841,E01018667,0.000000,L3. >70% suburban dwellers,1652,13.667057,2021-10-17-2021-12-12,North West,Cheshire East,Cheshire East 011,England
32842,E01015404,0.000000,L3. >70% suburban dwellers,1775,1.631246,2021-10-17-2021-12-12,South West,"Bournemouth, Christchurch and Poole",Poole 018,England


In [74]:
norm_area_df

,LSOA11CD,ALL_PEOPLE,Area
0,E01011954,2210,0.390643
1,E01011969,1293,0.838264
2,E01033465,1915,0.408588
3,E01011970,1108,0.367279
4,E01033467,2339,0.415572
...,...,...,...
32839,E01004738,1879,0.085649
32840,E01004665,1743,0.013524
32841,E01004669,1820,0.100576
32842,E01004739,1598,0.028525
